In [8]:
# page6

import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Data Generation
np.random.seed(42)
x = np.random.rand(100,1)
y = 1 + 2 * x + 1 * np.random.randn(100,1)

# Shuffles the indices
idx = np.arange(100)
np.random.shuffle(idx)

# Uses first 80 random indices for train
train_idx = idx[:80]

# Uses the remaining indices for validation
val_idx = idx[80:]

# Generation train and validation sets
x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val = x[val_idx], y[val_idx]

# NumPy 배열을 PyTorch 텐서로 변환하여 GPU로 이동 (필요시)
x_train_tensor = torch.from_numpy(x_train).float().to(device)
y_train_tensor = torch.from_numpy(y_train).float().to(device)
x_val_tensor = torch.from_numpy(x_val).float().to(device) # x_val 텐서 추가
y_val_tensor = torch.from_numpy(y_val).float().to(device) # y_val 텐서 추가

# DataLoader 생성
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True) # 배치 크기 설정

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(dataset=val_dataset, batch_size=16, shuffle=False) # 검증셋은 섞지 않음

# 모델 정의 (nn.Linear는 가중치와 편향을 내부적으로 관리합니다)
model_seq = nn.Sequential(nn.Linear(in_features=1, out_features=1)).to(device)

print(f"초기 모델 상태: {model_seq.state_dict()}") # 초기 가중치와 편향 확인

# 손실 함수 정의
loss_fn = nn.MSELoss(reduction="mean")

# 옵티마이저 정의: model_seq의 매개변수들을 최적화 대상으로 지정
optimizer = optim.SGD(model_seq.parameters(), lr=1e-1) # lr = 1e-1

# 학습 스텝 함수 정의
def make_train_step(model, loss_fn, optimizer):
  def train_step(x,y):
    model.train() # 모델을 훈련 모드로 설정

    yhat = model(x) # 예측

    loss = loss_fn(yhat, y) # 손실 계산
    loss.backward() # 역전파: 기울기 계산

    optimizer.step() # 매개변수 업데이트
    optimizer.zero_grad() # 다음 배치를 위해 기울기 초기화

    return loss.item() # 손실 값 반환
  return train_step

train_step = make_train_step(model_seq, loss_fn, optimizer)
losses = [] # 훈련 손실 저장 리스트
val_losses = [] # 검증 손실 저장 리스트 (추가)

n_epochs = 1000 # 에포크 수

for epoch in range(n_epochs):
  batch_losses = []
  for x_batch, y_batch in train_loader: # train_loader 사용
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    loss = train_step(x_batch, y_batch)
    batch_losses.append(loss)
  losses.append(np.mean(batch_losses)) # 에포크별 평균 훈련 손실 저장

  # 검증 단계 (torch.no_grad()로 기울기 계산 비활성화)
  with torch.no_grad():
    val_batch_losses = []
    for x_val_batch, y_val_batch in val_loader: # val_loader 사용
      x_val_batch = x_val_batch.to(device)
      y_val_batch = y_val_batch.to(device)

      model_seq.eval() # 모델을 평가 모드로 설정 (드롭아웃/배치 정규화 등에 영향)

      yhat_val = model_seq(x_val_batch) # x_val_batch만 입력
      val_loss = loss_fn(yhat_val, y_val_batch)
      val_batch_losses.append(val_loss.item())
    val_losses.append(np.mean(val_batch_losses)) # 에포크별 평균 검증 손실 저장

  # 진행 상황 출력 (선택 사항)
  if (epoch + 1) % 100 == 0:
    print(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}")

# 학습 후 모델의 최종 가중치와 편향 확인
print(f"학습 후 모델 상태: {model_seq.state_dict()}")


초기 모델 상태: OrderedDict([('0.weight', tensor([[0.8749]])), ('0.bias', tensor([-0.1011]))])
Epoch 100/1000, Train Loss: 0.8121, Val Loss: 0.5856
Epoch 200/1000, Train Loss: 0.8244, Val Loss: 0.5676
Epoch 300/1000, Train Loss: 0.8185, Val Loss: 0.5510
Epoch 400/1000, Train Loss: 0.8081, Val Loss: 0.5695
Epoch 500/1000, Train Loss: 0.8178, Val Loss: 0.5407
Epoch 600/1000, Train Loss: 0.8108, Val Loss: 0.5562
Epoch 700/1000, Train Loss: 0.8150, Val Loss: 0.5401
Epoch 800/1000, Train Loss: 0.8080, Val Loss: 0.5589
Epoch 900/1000, Train Loss: 0.8190, Val Loss: 0.5295
Epoch 1000/1000, Train Loss: 0.8059, Val Loss: 0.5505
학습 후 모델 상태: OrderedDict([('0.weight', tensor([[1.6793]])), ('0.bias', tensor([1.2268]))])
